<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2023/blob/main/dynamic_distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.integrate import solve_ivp
from plotly.subplots import make_subplots

In [17]:
alpha=2.
x=np.linspace(0,1,100)
y=x*alpha/(1-x+x*alpha)
fig=make_subplots()
fig.add_scatter(x=x,y=y,mode='lines')
fig.update_layout(width=600,height=600, template='plotly_dark')

In [91]:
NT=6
NF=3
weir_constant = 2.
F = 1.
zF = 0.6
q = 0.5

D = 0.55
R = 2

B=F-D

Kc = 10.
tray_m = 0.1  #molar holdup on tray
reboiler_m_sp = 0.5
condenser_m = 0.5

L_initial = np.r_[B, np.repeat(Vstrip + B, NF), np.repeat(Vrec - D, NT-NF+1)]

m_initial = np.r_[reboiler_m_sp, tray_m+(L_initial[1:-1]/weir_constant)**(2/3), condenser_m]

tend=200.

In [92]:
def rhs(t,vec,R):

    Vrec = (R+1)*D
    Vstrip = Vrec- (1-q)*F
    V=np.r_[np.repeat(Vstrip,NF), np.repeat(Vrec, NT-NF+1), 0.] #total condenser
    m, x = np.split(vec,2)
    y=x*alpha/(1-x+x*alpha)

    dm=np.zeros_like(m)
    dx = np.zeros_like(x)
    dmx=np.zeros_like(m)
    L = np.zeros_like(m)

    L[-1] = R*D
    L[0] =  F-D + Kc*(m[0]- reboiler_m_sp)

    L[1:-1] = weir_constant*(m[1:-1] - tray_m)**1.5
    dm[1:-1] = V[:-2]+L[2:] - V[1:-1] - L[1:-1]
    dmx[1:-1]= y[:-2]*V[:-2] + x[2:]*L[2:] - y[1:-1]*V[1:-1] - x[1:-1]*L[1:-1]

    dm[NF]+=F
    dmx[NF]+=zF*F

    dm[-1] = V[-2] - L[-1] - D
    dmx[-1]= y[-2]*V[-2] - x[-1]*L[-1] - x[-1]*D

    dm[0] = L[1] - V[0] - L[0]
    dmx[0]= x[1]*L[1] - y[0]*V[0] - x[0]*L[0]

    dx = (dmx- x*dm )/m

    return np.r_[dm, dx]


In [93]:
res=solve_ivp(lambda t,v: rhs(t,v,2.), (0,tend), np.r_[m_initial,np.repeat(zF,NT+2)], method='Radau',dense_output=True)
tplot = np.linspace(0,tend,101)
m, x=res.sol(tplot).reshape(2,NT+2,-1)
fig=make_subplots(rows=1,cols=2)

for i in range(NT+2):
    fig.add_scatter(x=tplot,y=x[i],row=1,col=1, name=f'x{i}')
    fig.add_scatter(x=tplot,y=m[i], row=1,col=2, name=f'm{i}')

fig.update_layout(width=1000,height=500,template='plotly_dark')

In [94]:
mcx=x[:,-1]
mcy=mcx*alpha/(1-mcx+mcx*alpha)
mcx=np.repeat(mcx,2)
mcy=np.repeat(mcy,2)
mcy[1:]=mcy[:-1]
mc = np.c_[mcx, mcy]
mc[0,1] = mc[0,0]
mc=mc[:-1]
alpha=2.
eqx=np.linspace(0,1,100)
eqy=eqx*alpha/(1-eqx+eqx*alpha)
fig=make_subplots()
fig.add_scatter(x=eqx,y=eqy,mode='lines')
fig.add_scatter(x=mc[:,0],y=mc[:,1],mode='lines',line_color='rgb(0,255,0)')
fig.add_scatter(x=mc[::2,0],y=mc[::2,1],mode='lines',line_color='rgb(255,0,0)')
fig.update_layout(width=600,height=600, template='plotly_dark')

In [95]:
res=solve_ivp(lambda t,v: rhs(t,v,10.), (0,tend), res.y[:,-1], method='Radau',dense_output=True)
tplot = np.linspace(0,tend,101)
m, x=res.sol(tplot).reshape(2,NT+2,-1)
fig=make_subplots(rows=1,cols=2)

for i in range(NT+2):
    fig.add_scatter(x=tplot,y=x[i],row=1,col=1, name=f'x{i}')
    fig.add_scatter(x=tplot,y=m[i], row=1,col=2, name=f'm{i}')

fig.update_layout(width=1000,height=500,template='plotly_dark')

In [96]:
mcx=x[:,-1]
mcy=mcx*alpha/(1-mcx+mcx*alpha)
mcx=np.repeat(mcx,2)
mcy=np.repeat(mcy,2)
mcy[1:]=mcy[:-1]
mc = np.c_[mcx, mcy]
mc[0,1] = mc[0,0]
mc=mc[:-1]
alpha=2.
eqx=np.linspace(0,1,100)
eqy=eqx*alpha/(1-eqx+eqx*alpha)
fig=make_subplots()
fig.add_scatter(x=eqx,y=eqy,mode='lines')
fig.add_scatter(x=mc[:,0],y=mc[:,1],mode='lines',line_color='rgb(0,255,0)')
fig.add_scatter(x=mc[::2,0],y=mc[::2,1],mode='lines',line_color='rgb(255,0,0)')
fig.update_layout(width=600,height=600, template='plotly_dark')